   ####  - Adding distance travelled
   ####  - Adding address

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from geopy.geocoders import Nominatim
import math as math

In [22]:
geolocator = Nominatim(user_agent="specify_your_app_name_here")

In [51]:
location = geolocator.reverse("40.719357 -74.001737")

In [52]:
print(location.address)

The Oltarsh Building, Canal Street, Chinatown, Manhattan, Manhattan Community Board 2, New York County, NYC, New York, 10013, USA


In [53]:
print(location.raw)

{'place_id': '135889691', 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': '248771081', 'lat': '40.71940195', 'lon': '-74.0014097410099', 'display_name': 'The Oltarsh Building, Canal Street, Chinatown, Manhattan, Manhattan Community Board 2, New York County, NYC, New York, 10013, USA', 'address': {'building': 'The Oltarsh Building', 'road': 'Canal Street', 'suburb': 'Chinatown', 'city_district': 'Manhattan', 'city': 'NYC', 'county': 'New York County', 'state': 'New York', 'postcode': '10013', 'country': 'USA', 'country_code': 'us'}, 'boundingbox': ['40.719194', '40.7195775', '-74.0016764', '-74.0011233']}


In [35]:
print(location.raw.get('address'))

{'building': '335 Madison Avenue', 'house_number': '335', 'road': 'Madison Avenue', 'suburb': 'Midtown East', 'city_district': 'Manhattan', 'city': 'NYC', 'county': 'New York County', 'state': 'New York', 'postcode': '10017', 'country': 'USA', 'country_code': 'us'}


In [54]:
print(location.raw.get('address').get('building'))
print(location.raw.get('address').get('house_number'))
print(location.raw.get('address').get('road'))
print(location.raw.get('address').get('suburb'))
print(location.raw.get('address').get('city_district'))

The Oltarsh Building
None
Canal Street
Chinatown
Manhattan


###  Merging data to create pic up and drop of lat_long

In [3]:
#importing data
train = pd.read_csv("C:/Users/shabbir.hasan/Documents/Hasan/Anaconda_rep/Kaggle/New_York_City_Taxi_Fare_Prediction/generated_datasets/clean_2.csv")

In [4]:
train['pickup_latitude'] = train['pickup_latitude'].round(6)
train['pickup_longitude'] = train['pickup_longitude'].round(6)
train['dropoff_latitude'] = train['dropoff_latitude'].round(6)
train['dropoff_longitude'] = train['dropoff_longitude'].round(6)

In [5]:
#removing lat and long with 0.0 values
train = train[train['pickup_latitude'] != 0.0]
train = train[train['pickup_longitude'] != 0.0]
train = train[train['dropoff_latitude'] != 0.0]
train = train[train['dropoff_longitude'] != 0.0]

In [6]:
#pickup lat and long
train['pic_lat_long'] = train['pickup_latitude'].astype(str)+ ' ' + train['pickup_longitude'].astype(str)

#drop lat and long
train['drop_lat_long'] = train['dropoff_latitude'].astype(str)+ ' ' + train['dropoff_longitude'].astype(str)

In [7]:
train.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day_of_month,date,day_of_week,weekend_weekday,hour,time_30_min_bin,time_15_min_bin,pic_lat_long,drop_lat_long
0,4.0,-73.994102,40.751041,-73.991257,40.754959,1,2015,3,10,2015-03-10,Tuesday,0,7,30,45,40.751041 -73.994102,40.754959 -73.991257
1,6.1,-73.961686,40.713017,-73.952406,40.699525,1,2011,12,17,2011-12-17,Saturday,1,0,30,45,40.713017 -73.961686,40.699525 -73.952406
2,9.7,-73.978325,40.753808,-73.985068,40.756177,1,2010,9,16,2010-09-16,Thursday,0,0,0,15,40.753808 -73.978325,40.756177 -73.985068
3,6.9,-74.001737,40.719357,-73.992353,40.740713,1,2012,6,13,2012-06-13,Wednesday,0,18,0,0,40.719357 -74.001737,40.740713 -73.992353
4,6.0,-73.966393,40.770531,-73.958328,40.781441,3,2015,2,4,2015-02-04,Wednesday,0,7,30,30,40.770531 -73.966393,40.781441 -73.958328


#### calculating radial distance

In [10]:
def distance(lat1 , lon1, lat2, lon2):
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return round(d,2)

In [11]:
#test
distance(40.75104,-73.994102,40.75104,-73.994102)

0.0

In [12]:
train['distance_travelled'] = train.apply(lambda x: distance(x['pickup_latitude'], x['pickup_longitude'], 
                                           x['dropoff_latitude'], x['dropoff_longitude']), axis=1)

In [15]:
train.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day_of_month,date,day_of_week,weekend_weekday,hour,time_30_min_bin,time_15_min_bin,pic_lat_long,drop_lat_long,distance_travelled
0,4.0,-73.994102,40.751041,-73.991257,40.754959,1,2015,3,10,2015-03-10,Tuesday,0,7,30,45,40.751041 -73.994102,40.754959 -73.991257,0.50
1,6.1,-73.961686,40.713017,-73.952406,40.699525,1,2011,12,17,2011-12-17,Saturday,1,0,30,45,40.713017 -73.961686,40.699525 -73.952406,1.69
2,9.7,-73.978325,40.753808,-73.985068,40.756177,1,2010,9,16,2010-09-16,Thursday,0,0,0,15,40.753808 -73.978325,40.756177 -73.985068,0.63
3,6.9,-74.001737,40.719357,-73.992353,40.740713,1,2012,6,13,2012-06-13,Wednesday,0,18,0,0,40.719357 -74.001737,40.740713 -73.992353,2.50
4,6.0,-73.966393,40.770531,-73.958328,40.781441,3,2015,2,4,2015-02-04,Wednesday,0,7,30,30,40.770531 -73.966393,40.781441 -73.958328,1.39


In [ ]:
#calc google dist

# import googlemaps 
# # Requires API key 
# gmaps = googlemaps.Client(key='Your_API_key')  
# # Requires cities name 
# distance = gmaps.distance_matrix('Delhi','Mumbai')['rows'][0]['elements'][0]

# print(distance)
# {u'distance': {u'text': u'1,418 km', u'value': 1417632},
#  u'duration': {u'text': u'1 day 0 hours', u'value': 87010},
#  u'status': u'OK'}
# If you have geo coordinates(latitude and longitude) of origin and destination, then do as below:

# # Requires geo-coordinates(latitude/longitude) of origin and destination
# origin_latitude = 12.9551779
# origin_longitude = 77.6910334
# destination_latitude = 28.505278
# destination_longitude = 77.327774
# distance = gmaps.distance_matrix([str(origin_latitude) + " " + str(origin_longitude)], [str(destination_latitude) + " " + str(destination_longitude)], mode='walking')['rows'][0]['elements'][0]

# print(distance)
# {u'distance': {u'text': u'2,014 km', u'value': 2013656},
#  u'duration': {u'text': u'16 days 23 hours', u'value': 1464529},
#  u'status': u'OK'}

###  getting address

In [26]:
train.dtypes

fare_amount          float64
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
year                   int64
month                  int64
day_of_month           int64
date                  object
day_of_week           object
weekend_weekday        int64
hour                   int64
time_30_min_bin        int64
time_15_min_bin        int64
pic_lat_long          object
drop_lat_long         object
dtype: object

In [64]:
def geo_loc(x):
    geolocator = Nominatim(user_agent="specify_your_app_name_here")
    location = geolocator.reverse(x)
    
    #buiding
    if location.raw.get('address').get('building'):
        building = location.raw.get('address').get('building')
    else:
        building = "notProvided"
    
    # houseNumber
    if location.raw.get('address').get('house_number'):
        house_number = location.raw.get('address').get('house_number')
    else:
        house_number = "notProvided"
    
    #road
    if location.raw.get('address').get('road'):
        road = location.raw.get('address').get('road')
    else:
        road = "notProvided"

    #suburb
    if location.raw.get('address').get('suburb'):
        suburb = location.raw.get('address').get('suburb')
    else:
        suburb = "notProvided"
        
    #city_district
    if location.raw.get('address').get('city_district'):
        city_district = location.raw.get('address').get('city_district')
    else:
        city_district = "notProvided"
            
    address = building + " | " + house_number + " | " + road + " | " + suburb + " | " + city_district
    return address


In [68]:
geo_loc("40.719357 -74.001737")

'The Oltarsh Building | notProvided | Canal Street | Chinatown | Manhattan'

In [69]:
train.shape

(10863680, 18)

In [72]:
#train['pic_address'] = train.pic_lat_long.apply(lambda x: geo_loc(x.decode('utf-8')))
train['pic_full_address'] = train.pic_lat_long.apply(lambda x: geo_loc(x))

# train.head().pic_lat_long.apply(lambda x: geo_loc(x))

GeocoderTimedOut: Service timed out

In [ ]:
train['drop_full_address'] = train.drop_lat_long.apply(lambda x: geo_loc(x))

In [17]:
train.to_csv('clean_4_with_geoLoc.csv', index=False )

In [73]:
from pygeocoder import Geocoder

In [77]:
results = Geocoder.reverse_geocode(40.719357, -74.001737)

GeocoderError: Error REQUEST_DENIED
Query: https://maps.google.com/maps/api/geocode/json?latlng=40.719357%2C-74.001737&sensor=false&bounds=&region=&language=